In [1]:
import os
import sys
import pandas
import numpy
import shutil
import cProfile, pstats, io

%load_ext cython

In [2]:
from package_algorithms.sat.sa_sat import SimulatedAnnealing_SAT
from package_parsers.sat_parser import SATParser
from algorithm_tester_common.tester_dataclasses import AlgTesterContext
from algorithm_tester.helpers import create_path
from algorithm_tester.concurrency_runners import BaseRunner

In [3]:
def profile(fnc):
    
    """A decorator that uses cProfile to profile a function"""
    
    def inner(*args, **kwargs):
        
        pr = cProfile.Profile()
        pr.enable()
        retval = fnc(*args, **kwargs)
        pr.disable()
        s = io.StringIO()
        sortby = 'cumulative'
        ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
        ps.print_stats()
        print(s.getvalue())
        return retval

    return inner

In [4]:
@profile
def profileRunAlgorithms(context, instance_data):
    sa = SimulatedAnnealing_SAT()
    sa.perform_algorithm(context, instance_data)

parser: SATParser = SATParser()

out_dir: str = ".tmp_out_dir"
context: AlgTesterContext = AlgTesterContext(
    algorithms=["SA_SAT"], parser="SATParser", communicators=[], concurrency_runner=BaseRunner(),
    check_time=True, time_retries=1, max_num=None,
    extra_options={"init_temperature":1000, "min_temperature":1, "cooling":0.995, "cycles":50, "create_evo_file":False},
    input_dir="../data/NK", output_dir=out_dir
    )
    
create_path(out_dir)
with open("../data/wuf-A/wuf20-88-A/wuf20-01000-A.mwcnf") as instance_file:
    instance_data = parser.get_next_instance(instance_file)
    profileRunAlgorithms(context, instance_data)
    
shutil.rmtree(out_dir)

         1213478 function calls in 15.790 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   15.790   15.790 <ipython-input-4-c7cd1fafaa66>:1(profileRunAlgorithms)
        1    0.000    0.000   15.790   15.790 /Users/petr/Documents/Projects/Python/AlgorithmTester/package_algorithms/sat/sa_sat.py:173(perform_algorithm)
        1    0.151    0.151   15.790   15.790 /Users/petr/Documents/Projects/Python/AlgorithmTester/package_algorithms/sat/sa_sat.py:133(get_solution)
    68950    0.227    0.000   15.349    0.000 /Users/petr/Documents/Projects/Python/AlgorithmTester/package_algorithms/sat/sa_sat.py:90(get_new_neighbour)
    68951   14.928    0.000   14.928    0.000 {built-in method csa_sat.check_validity}
    68950    0.103    0.000    0.246    0.000 /Users/petr/Documents/Projects/Python/AlgorithmTester/package_algorithms/sat/alg_dataclasses.py:40(copy)
    68950    0.030    0.000    0.193    0.00

In [5]:
%%cython

import time
import random
from libc.math cimport exp
from libc.stdlib cimport rand, srand, RAND_MAX
from libc.stdio cimport fopen, FILE, fclose, fprintf

cpdef void set_seed(int seed):
    srand(seed)
    
cpdef float random_float():
    return rand()/RAND_MAX

cpdef int random_int(int low = 0, int height = 2):
    return rand() % height + low
            
cpdef test(str path):
    with open(path, "w") as out:
        print(type(out))
        out.write("test\n")

In [6]:

    
test("test_file.csv")

<class '_io.TextIOWrapper'>


In [7]:
cooling = 0.995
max_temp = 1000
min_temp = 0.5
cycles = 50
cntr = 0

while max_temp > min_temp:
    for _ in range(cycles):
        cntr += 1
    max_temp *= cooling

cntr

75850

In [8]:
numpy.array([[1, 2], [3, 4], [5, 6, 7]])

array([list([1, 2]), list([3, 4]), list([5, 6, 7])], dtype=object)